## General UEDGE things
Plotting 2D and dimensions
Downloading/installing/building
Recommend using pyenvs and pyenv-virtualvenvs

Display use of:
- nx, ny
- Case.get
- shape
- about
- search
- searchvarname

- ixpt1
- ixpt2
- iysptrx

- Structure of ion arrays
- Structure of gaseous arrays

- Node structure

- Save variables
- Populating (UEDGE and Case.populate)

- Saving and restoring
- Tracking changes to inputs

- Defining YAML variables to be saved



# Getting started with the UEDGE Toolbox

Welcome to the introduction of UETOOLS, short for the UEDGE Toolbox. The classes provided by UETOOLS interact with UEDGE on a fundamental level, and aims to provide the user with powerful tools to for running and processing UEDGE simulations. This guide will take you through the basics of UETOOLS. Some aspects of UEDGE functionality, important for understanding the functionality of the UETOOLS, will be presented briefly here, but more in-depth discussion of UEDGE itself is reserved for the UEDGE documentation itself. 

## TOC:
* [Installing UETOOLS](#installing-uetools)
* [Installing UEDGE](#installing-UEDGE)
* [Structure of UETOOLS](#structure)

## Installing UETOOLS<a class="anchor" id="installing-uetools"></a>
Presently, UETOOLS can be installed by cloning the [UETOOLS GitHub repository](https://www.github.com/llnl/uetools) and including the location of UETOOLS in your \$PATH variable. More install options are coming shortly.

## Installing UEDGE<a class="anchor" id="installing-uedge"></a>
UEDGE is a prerequisite for UETOOLS. Since UEDGE can be installed by pip, or built locally, it is not, however, a forced prerequisite for installing UETOOLS. E.g. the user will need to ensure a functioning version of UEDGE is available to the Python command prompt. The most straightforward way to install UEDGE is using pip install
    pip install uedge
For more information on installing UEDGE, consult the UEDGE manual.

## Structure of UETOOLS<a class="anchor" id="structure"></a>
UETOOLS provides the user with two primary classes for interacting with UEDGE case: the Case and Database classes. These are presented shhortly below.

### The Case Class
Case is the primary class in UETOOLS, and handles most of the setup of a UEDGE case, and provides an interface for creating YAML input files for UEDGE. The base Case class inherits methods from several classes, making it very modular and easy to patch to the individual user needs. The Case class itself primarily handles the exchange of information with UEDGE, such as changing or getting variable values, setting up a case based on the input file, and restoring saved solutions. Here follows a short description of the most commonly used classes, which Case inherits:

- Caseplot: handles specific plotting, using the high-level plotting functions avilable in the Plot class.
- Save: handles saving of data to UETOOLS save-files, compatible the with UEDGE restore function. The UETOOLS save-files also store additional data, such as grid data, and some commonly used derived variables. Thus, UETOOLS can restore UEDGE cases from save-files only, removing the need for separate input and grid files. Additionally, the saves can be used as data dumps as discussed later.
- Solver: provides aliases for the UEDGE solvers through the Case class
- PostProcessors: contains a number of vetted and approved post-processors for numerous UEDGE quantities

### The Database Class
Because only a single instance of UEDGE can be run in a Python prompt, UETOOLS provides a database class that can statically read data from UETOOLS save-files and compile them to databases that can be scanned/compared. Using a single Python prompt to restore many UEDGE cases in succession is to be avoided, as there may be issues with the grid sizes, or different setups can make modifications to variables which are not reset in subsequent files, making the results order-dependent. The database instead creates static UETOOLS Case objects that are stored in array, allowing data to be accessed for all cases stored, simultaneously.

The use of databases is discussed in greater detail below.

## Running UETOOLS for the first time
When creating a Case object, UETOOLS checks whether a configuration file for UETOOLS is present. If no configuration file is present, a message is raised:

In [1]:
from uetools import Case
c=Case()

No UETOOLS config file found: Configure file by calling Case.CreateConfig()
Alternatively, manually create the .uetoolsrc configuration YAML in your home directory.


The configuration file can be created using UETOOLS:

In [15]:
c.CreateConfig()

UEDGE config file not found!
Create it at /Users/holm10? [y/n]


 y


Define path to "aphdir":


 ~/Documents/fusion/uedge/rates/aph


    Path defined successfully!
Define path to "apidir":


 ~/Documents/fusion/uedge/rates/api


    Path defined successfully!
UEDGE config file /Users/holm10.uetoolsrc successfully created!


Alternativelly, the configuration file can be written manually into your home directory. If you would like to use UETOOLS without a configuration file, you can supress the startup message by creating an empty .uetoolsrc-file in your home directory.

## UEDGE dimensions and arrays
## The UEDGE grid
### The X-point cut